# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'colab_main_dataset' #'colab_main_dataset'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['lfw', 'scface',  'enfsi', 'enfsi2015'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

[20:01:45] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[20:01:45] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.8/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[20:01:45] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:01:45] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[20:01:45] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:01:45] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[20:01:45] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:01:45] ../src/nnvm/lega

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2022-12-11 20:01:45.971335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'colab_main_dataset',
 'input_dir': '/home/jmacarulla/video_resources/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/video_resources/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f8f795d9e50>,
 'databases': [<sql_face.databases.LFW at 0x7f8f795d9e80>,
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
database.create_tables(serfiq=serfiq)

In [7]:
database.update_tables(attributes_to_update, serfiq = serfiq)

Update gender: 0it [00:00, ?it/s]
Update gender: 0it [00:00, ?it/s]
Update gender: 0it [00:00, ?it/s]
Update gender: 0it [00:00, ?it/s]
Update cropped images: 0it [00:00, ?it/s]
Computing embeddings: 0it [00:00, ?it/s]
Computing SER-FIQ quality:   0%|          | 0/15077 [00:00<?, ?it/s]

1/1 [==============================] - 0s 251ms/step


Computing SER-FIQ quality:   0%|          | 1/15077 [08:53<2232:23:27, 533.07s/it]

1/1 [==============================] - 0s 44ms/step


Computing SER-FIQ quality:   0%|          | 2/15077 [15:03<1832:38:02, 437.64s/it]

...

1/1 [==============================] - 0s 31ms/step


Computing SER-FIQ quality:   1%|          | 77/15077 [27:36<43:33:13, 10.45s/it]

1/1 [==============================] - 0s 35ms/step


Computing SER-FIQ quality:   1%|          | 78/15077 [27:47<43:27:23, 10.43s/it]

1/1 [==============================] - 0s 35ms/step
